# Deep Learning Based Fusion Approach for Hate Speech Detection

# task


## Load Libraries

In [34]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import json
import copy
import re
import csv
from keras.utils import to_categorical
import codecs
import keras.preprocessing.text as kpt
from keras.preprocessing.sequence import pad_sequences
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation , LSTM , Input , Embedding
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, concatenate, Activation, Average
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout , Bidirectional
from keras.layers import Flatten , LSTM , Reshape, LeakyReLU
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.regularizers import L1L2
from keras import optimizers
from keras.callbacks import CSVLogger
import keras
import keras.preprocessing.text as kpt
#from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json

In [35]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [36]:
import os
os.chdir("C:/Users/M Srikant/Desktop/160118862036_DL_HPD/CODE/Original Code/Task_A/Data")
os.getcwd()

'C:\\Users\\M Srikant\\Desktop\\160118862036_DL_HPD\\CODE\\Original Code\\Task_A\\Data'

In [37]:
#train.tsv
train=pd.read_table('train_en.tsv',delimiter='\t',encoding='utf-8')
print(list(train.columns.values)) 

['id', 'text', 'HS', 'TR', 'AG']


In [5]:
train.head()

,id,text,HS,TR,AG
0,201,"Hurray, saving us $$$ in so many ways @potus @...",1,0,0
1,202,Why would young fighting age men be the vast m...,1,0,0
2,203,@KamalaHarris Illegals Dump their Kids at the ...,1,0,0
3,204,NY Times: 'Nearly All White' States Pose 'an A...,0,0,0
4,205,Orban in Brussels: European leaders are ignori...,0,0,0


In [39]:
train.count()

id      9000
text    9000
HS      9000
TR      9000
AG      9000
dtype: int64

In [40]:
my_df = train

In [41]:
print(my_df.text.head(2))

0    Hurray, saving us $$$ in so many ways @potus @...
1    Why would young fighting age men be the vast m...
Name: text, dtype: object


In [42]:
my_word_stop = ['the','in','of','is','a','to','an','be','are','for','was','it','as','on', 'so', 'at']


In [43]:
with open('segmentation_train_dev.csv', 'w') as f:
    writer = csv.writer(f)
    for row in my_df.text:
        row = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*","url",row)
        row = " ".join([word for word in row.split() if word not in my_word_stop])
        row = re.sub(r"(.)\1+", r"\1",row)
        rest_array = [text.encode("utf8") for text in row]
        rest_array = b"".join(rest_array)
        writer.writerow(rest_array)
f.close()

In [44]:
train=pd.read_table('segmentation_train_dev.csv',delimiter='\t',header=None)

In [45]:
train.count()

0    9000
dtype: int64

In [46]:
from keras.preprocessing.text import Tokenizer

In [47]:
train[0] = train[0].astype('str')

In [48]:
train_x = train[0]

In [49]:
train_y = my_df.HS

In [50]:
my_df.HS.count()

9000

In [51]:

tokenizer = Tokenizer(lower=False,filters=',',char_level=False)
tokenizer.fit_on_texts(train_x)

In [52]:

dictionary = tokenizer.word_index
with open('dictionary.p', 'wb') as fp:
    pickle.dump(dictionary, fp)


def convert_text_to_index_array(text):
    temp_wordIndices = []
    for word in kpt.text_to_word_sequence(text,filters=',',lower=False):
        if word in dictionary:
            temp_wordIndices.append(dictionary[word])
    return temp_wordIndices

allWordIndices = []
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

allWordIndices = np.asarray(allWordIndices)


C:\Users\M Srikant\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [53]:
print('Found %s unique tokens.' % len(dictionary))

Found 170 unique tokens.


In [54]:
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
train_y = to_categorical(train_y, 2)

In [55]:
train_x.shape[1]


171

In [56]:
from keras.layers.advanced_activations import LeakyReLU

In [57]:
model = Sequential()

model.add(Dense(100, activation='relu',input_shape=(train_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

print(model.summary())

filepath="sequencing_the_data_try_n_error.{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('final_log.csv', append=True, separator=';')

model.fit(train_x, train_y,
    batch_size=5,
    epochs=15,
    verbose=1,
    validation_split=0.20,
    shuffle=True,callbacks = [csv_logger])

model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

print('saved model!')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               17200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 402       
Total params: 37,802
Trainable params: 37,802
Non-trainable params: 0
_________________________________________________________________
None
Train on 7200 samples, validate on 1800 samples
Epoch 1/15
7200/7200 [==============================] - 7s 1ms/

In [25]:
#test.tsv
test=pd.read_table('dev_en.tsv',delimiter='\t',encoding='utf-8')
print(list(test.columns.values)) #file header

['id', 'text', 'HS', 'TR', 'AG']


In [26]:
test.head()

,id,text,HS,TR,AG
0,18201,I swear I’m getting to places just in the nick...,0,0,0
1,18202,I’m an immigrant — and Trump is right on immig...,0,0,0
2,18203,#IllegalImmigrants #IllegalAliens #ElectoralSy...,1,0,1
3,18204,@DRUDGE_REPORT We have our own invasion issues...,1,0,1
4,18205,Worker Charged With Sexually Molesting Eight C...,0,0,0


In [27]:
with open('segmentation_test.csv', 'w') as f:
    writer = csv.writer(f)
    for row in test.text:
        row = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*","url",row)
        row = " ".join([word for word in row.split() if word not in my_word_stop])
        row = re.sub(r"(.)\1+", r"\1",row)
        rest_array = [text.encode("utf8") for text in row]
        rest_array = b"".join(rest_array)
        writer.writerow(rest_array)
f.close()

In [28]:
test_final=pd.read_csv('segmentation_test.csv',delimiter='\t',header=None)
print(list(test_final.columns.values)) #file header
test_final[0] = test_final[0].astype('str')

[0]


In [29]:
rest_array[2]

110

In [30]:
tokenizer = Tokenizer(num_words=train_x.shape[1])


with open('dictionary.p', 'rb') as fp:
    dictionary = pickle.load(fp)
    
    
not_found_word_list = []
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text,filters='',lower=False,split=',')
    wordIndices = []
    no_word = 0
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            if word == "":
                not_found_word_list.append(word)
                no_word = no_word + 1
    
    return wordIndices,no_word

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('model.h5')
with open('fileName.csv', 'w') as f:
    count=0
    no_words = 0
    for row in test_final[0]:
        evalSentence = row
        testArr,no_word = convert_text_to_index_array(evalSentence)
        input = tokenizer.sequences_to_matrix([testArr], mode='binary')
        pred = model.predict(input)
        f.write(str(np.argmax(pred)) + "\n")
        count+=1
        no_words+=no_word
f.close()
print(count)
print("word not found : ", no_words)
print(count)
print("word found : ", wordIndices)
with open('not_found_word_list.csv', 'w') as f:
    for word in not_found_word_list:
        f.write(str(word)+"\n")
f.close()

1000
word not found :  0
1000
word found :  [36, 2, 3, 7, 10, 17, 1, 2, 24, 2, 7, 17, 1, 19, 8, 14, 3, 6, 1, 26, 1, 22, 6, 8, 19, 1, 12, 3, 9, 1, 28, 14, 2, 25, 8, 1, 4, 12, 2, 5, 7, 1, 21, 2, 13, 1, 52, 9, 2, 53, 11, 3, 10, 1, 12, 3, 7, 2, 9, 14, 2, 6, 4, 31, 3, 20, 11, 9, 2, 31, 7, 3, 18, 2, 1, 3, 10, 1, 52, 1, 17, 2, 9, 1, 5, 4, 32, 9, 1, 12, 2, 3, 7, 4, 20, 7, 2, 3, 22, 5, 6, 16, 31, 1, 17, 2, 9, 1, 4, 12, 2, 7, 2, 32, 9, 1, 14, 11, 15, 12, 1, 62, 1, 13, 8, 23]
